In [25]:
import torch
from torch import nn
from torch.autograd import Variable
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from LSTM import *
from LSTMbis import *
import pandas as pd
import numpy as np
import torch.utils.data as utils
import time
import os
import pickle

import pdb

# For the notebook
%matplotlib inline
%reload_ext autoreload
%autoreload 2

torch.manual_seed(1)

In [26]:
gt_test_coord_1 = pickle.load( open( "./import_dataset_2/test/gt_test_coord_1.pkl", "rb" ) )
gt_test_1 = pickle.load( open( "./import_dataset_2/test/gt_test_1.pkl", "rb" ) )
in_test_coord_1 = pickle.load( open( "./import_dataset_2/test/in_test_coord_1.pkl", "rb" ) )
inputs_test_1 = pickle.load( open( "./import_dataset_2/test/inputs_test_1.pkl", "rb" ) )

gt_test_coord_2 = pickle.load( open( "./import_dataset_2/test/gt_test_coord_2.pkl", "rb" ) )
gt_test_2 = pickle.load( open( "./import_dataset_2/test/gt_test_2.pkl", "rb" ) )
in_test_coord_2 = pickle.load( open( "./import_dataset_2/test/in_test_coord_2.pkl", "rb" ) )
inputs_test_2 = pickle.load( open( "./import_dataset_2/test/inputs_test_2.pkl", "rb" ) )

gt_test_coord_3 = pickle.load( open( "./import_dataset_2/test/gt_test_coord_3.pkl", "rb" ) )
gt_test_3 = pickle.load( open( "./import_dataset_2/test/gt_test_3.pkl", "rb" ) )
in_test_coord_3 = pickle.load( open( "./import_dataset_2/test/in_test_coord_3.pkl", "rb" ) )
inputs_test_3 = pickle.load( open( "./import_dataset_2/test/inputs_test_3.pkl", "rb" ) )

inputs_test_1 = torch.from_numpy(inputs_test_1).float()
gt_test_1 = torch.from_numpy(gt_test_1).float()

inputs_test_2 = torch.from_numpy(inputs_test_2).float()
gt_test_2 = torch.from_numpy(gt_test_2).float()

inputs_test_3 = torch.from_numpy(inputs_test_3).float()
gt_test_3 = torch.from_numpy(gt_test_3).float()
inputs_test_1.shape, inputs_test_2.shape, inputs_test_3.shape



(torch.Size([10, 251, 164]),
 torch.Size([10, 1007, 164]),
 torch.Size([10, 460, 164]))

In [29]:
coord_ind = np.append(np.arange(0,2),np.arange(4,164))
inputs_test_1 = inputs_test_1[:,:,coord_ind]
gt_test_1 = gt_test_1[:,:,:2]
inputs_test_2 = inputs_test_2[:,:,coord_ind]
gt_test_2 = gt_test_2[:,:,:2]
inputs_test_3 = inputs_test_3[:,:,coord_ind]
gt_test_3 = gt_test_3[:,:,:2]
inputs_test_1.shape, inputs_test_2.shape, inputs_test_3.shape


(torch.Size([10, 251, 162]),
 torch.Size([10, 1007, 162]),
 torch.Size([10, 460, 162]))

##### Full LSTM

In [30]:
full_lstm = LSTM(input_size = inputs_test_1.shape[2], output_size = 2,
                 num_layers=2, hidden_size=128)
full_lstm.load_state_dict(torch.load('./model_rodolphe/model_results/coord_1.pt'))


In [33]:
output_test_1 = full_lstm.predict(inputs_test_1)
output_test_1 = output_test_1.data.numpy()

output_test_2 = full_lstm.predict(inputs_test_2)
output_test_2 = output_test_2.data.numpy()

output_test_3 = full_lstm.predict(inputs_test_3)
output_test_3 = output_test_3.data.numpy()
output_test_1.shape

(10, 251, 2)

In [35]:
output_coord_1=np.zeros([11,inputs_test_1.shape[1],2])
for j in range(11):
    for i in range(inputs_test_1.shape[1]):
        if j==0:
            output_coord_1[j,i,0:2] = in_test_coord_1[9,i,0:2]
        else:
            output_coord_1[j,i,0:2] = output_test_1[j-1,i,:2]#*0.4 + output_coord_1[j-1,i,0:2]


In [37]:
dist = np.zeros(inputs_test_1.shape[1])
for i in range(inputs_test_1.shape[1]):
    dist[i] = np.sqrt(sum((gt_test_coord_1[10,i,:]-output_coord_1[10,i,:])**2))
    
final_coord_error = np.mean(dist)
print('The final distance between the ground trought and the predicted coordinates is :',
      final_coord_error.round(3))

avr = np.zeros((inputs_test_1.shape[1],11))
for i in range(inputs_test_1.shape[1]):
    for j in range(11):
        avr[i,j] = np.sqrt(sum((gt_test_coord_1[j,i,:]-output_coord_1[j,i,:])**2))

average = np.mean(np.mean(avr,1))
print('The average error between the ground trought and the predicted coordinates is :',
      average.round(3))        

The final distance between the ground trought and the predicted coordinates is : 1.269
The average error between the ground trought and the predicted coordinates is : 1.16


In [22]:
output_coord_2=np.zeros([11,inputs_test_2.shape[1],2])
for j in range(11):
    for i in range(inputs_test_2.shape[1]):
        if j==0:
            output_coord_2[j,i,0:2] = in_test_coord_2[9,i,0:2]
        else:
            output_coord_2[j,i,0:2] = output_coord_2[j-1,i,0:2]+output_test_2[j-1,i,2:]*0.4
        

dist = np.zeros(inputs_test_2.shape[1])
for i in range(inputs_test_2.shape[1]):
    dist[i] = np.sqrt(sum((gt_test_coord_2[10,i,:]-output_coord_2[10,i,:])**2))

final_coord_error = np.mean(dist)
print('The final distance between the ground truth and the predicted coordinates is :',final_coord_error.round(3))

avr = np.zeros((inputs_test_2.shape[1],11))
for i in range(inputs_test_2.shape[1]):
    for j in range(11):
        avr[i,j] = np.sqrt(sum((gt_test_coord_2[j,i,:]-output_coord_2[j,i,:])**2))

average = np.mean(np.mean(avr,1))
print('The average error between the ground truth and the predicted coordinates is :',average.round(3))

The final distance between the ground truth and the predicted coordinates is : 0.871
The average error between the ground truth and the predicted coordinates is : 0.484


In [23]:
output_coord_3=np.zeros([11,inputs_test_3.shape[1],2])
for j in range(11):
    for i in range(inputs_test_3.shape[1]):
        if j==0:
            output_coord_3[j,i,0:2] = in_test_coord_3[9,i,0:2]
        else:
            output_coord_3[j,i,0:2] = output_coord_3[j-1,i,0:2]+output_test_3[j-1,i,2:]*0.4
        

dist = np.zeros(inputs_test_3.shape[1])
for i in range(inputs_test_3.shape[1]):
    dist[i] = np.sqrt(sum((gt_test_coord_3[10,i,:]-output_coord_3[10,i,:])**2))

final_coord_error = np.mean(dist)
print('The final distance between the ground truth and the predicted coordinates is :',
      final_coord_error.round(3))


avr = np.zeros((inputs_test_3.shape[1],11))
for i in range(inputs_test_3.shape[1]):
    for j in range(11):
        avr[i,j] = np.sqrt(sum((gt_test_coord_3[j,i,:]-output_coord_3[j,i,:])**2))

average = np.mean(np.mean(avr,1))
print('The average error between the ground trought and the predicted coordinates is :',
      average.round(3))

The final distance between the ground truth and the predicted coordinates is : 1.093
The average error between the ground trought and the predicted coordinates is : 0.568
